In [1]:
import tensorflow as tf
from time import time
import numpy
import os
import json
import pickle
import pandas
import datetime
from functools import partial, reduce

import sys
sys.path.append('../libs')

import data_pipeline
import conv_model
import initialize
import prepare_data
import flacdb

tf.debugging.set_log_device_placement(True)

### Plot Batch

In [2]:
H0 = initialize.load_hypes()
H = {
    **H0, 
    'input_sigs_train': H0['output_sigs'] + H0['input_sigs_train'], 
    'input_sigs_validation': H0['output_sigs'] + H0['input_sigs_validation'], 
    'epochs': 1,
    'steps_per_epoch': 32,
    'window_size': 512,
    'batch_buffer_size': 1,
}

In [3]:
%time initialize.run(H)

132 years,  594510 record segments


/sailhome/kuprel/blood-pressure/env3p7/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/sailhome/kuprel/blood-pressure/env3p7/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 1min 15s, sys: 10.4 s, total: 1min 25s
Wall time: 1min 25s


In [3]:
%time data = initialize.load_data(H)

Executing op Cast in device /job:localhost/replica:0/task:0/device:CPU:0
CPU times: user 2.12 s, sys: 484 ms, total: 2.6 s
Wall time: 2.62 s


In [4]:
dataset = data_pipeline.build(H, data['train'], 'train')
x_, y_ = next(iter(dataset))
x, y = x_.numpy(), y_['pressure'].numpy()

Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Transpose in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op BroadcastTo in device /job:localhost/replica:0/task:0/device:GPU:0


In [11]:
%matplotlib widget
from matplotlib import pyplot
from functools import partial
from ipywidgets import interact, IntSlider


S_in, S_out = H0['input_sigs_train'], H0['output_sigs']
S = S_out + S_in

bp_sigs = ['ABP', 'CVP', 'PAP', 'ICP', 'ART']
ecg_sigs = ['I', 'II', 'III', 'V', 'AVR', 'AVF', 'AVL', 'MCL', 'MCL1']

sig_colors = {
    **{k: 'red' for k in bp_sigs},
    **{k: 'blue' for k in ecg_sigs},
    'RESP': 'green',
    'PLETH': 'green',
}

def plot_batch_example(H, x, y, i):
    pyplot.subplots_adjust(left=0.03, wspace=0, hspace=0.2)
    lines = {'sigs': {}, 'label': {}}
    axes = {}
    
    def plot_y(s, j, is_diastolic):
        xs = [0, H['window_size']]
        ys = [y[i, int(is_diastolic), j]] * 2
        line = axes[s].plot(xs, ys, '--', c='gray')[0]
        return line
    
    for j, s in enumerate(S):
        axes[s] = pyplot.subplot(len(S), 1, j + 1)
        line = axes[s].plot(x[i, :, j])[0]
        lines['sigs'][s] = line
        axes[s].set_ylabel(s)
        axes[s].set_facecolor('black')
        axes[s].yaxis.tick_right()
        if s in bp_sigs:
            lines['label'][s] = {
                'sys': plot_y(s, j, is_diastolic=False), 
                'dia': plot_y(s, j, is_diastolic=True)
            }
    
    return lines, axes

def update(i):
    for j, s in enumerate(S):
        lines['sigs'][s].set_ydata(x[i, :, j])
        low, high = x[i, :, j].min(), x[i, :, j].max()
        dx = max(0.01, high - low)
        sig_color = sig_colors[s] if x[i, :, j].any() else 'black'
        lines['sigs'][s].set_color(sig_color)
        axes[s].set_ylim(bottom = low - 0.1 * dx, top = high + 0.1 * dx)
        if s in S_out:
            line = lines['label'][s]
            line['sys'].set_ydata([y[i, 0, j]] * 2)
            line['dia'].set_ydata([y[i, 1, j]] * 2)
            line['sys'].set_color('gray' if y[i, :, j].any() else 'black')
            line['dia'].set_color('gray' if y[i, :, j].any() else 'black')
        
        axis = fig.axes[j]
        axis.yaxis.label.set_color('white' if x[i, :, j].any() else 'dimgray')
        axis.tick_params(axis='y', colors='gray')
        if x[i, :, j].any():
            ticks = [x[i, :, j].min(), x[i, :, j].max()]
        else:
            ticks = []
        axis.set_yticks(ticks)
        
    fig.canvas.draw()
    pyplot.show()


fig = pyplot.figure(facecolor='black')
fig.canvas.layout.height = '800px'
fig.canvas.layout.width = '700px'
lines, axes = plot_batch_example(H, x, y, 0)

interact(update, i=IntSlider(min=0, max=H['batch_size']-1, value=0));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='i', max=63), Output()), _dom_classes=('widget-interact',…

### Filter Percentage

In [ ]:
%%time

N = 10**4

H_ = {
    **H, 
    'epochs': 1,
    'steps_per_epoch': N / H['batch_size'] / 10 * 2,
    'batch_buffer_size': 1,
    'windows_per_chunk': 10,
    'filter_data': True
}

dataframe = initialize.sample_data(H_, sig_data)
print(dataframe.shape)
I = numpy.random.permutation(dataframe.shape[0] // H_['windows_per_chunk'])[:N // H_['windows_per_chunk']]
I = [i*H_['windows_per_chunk'] + j for i in I for j in range(H_['windows_per_chunk'])]
tensors = initialize.dataframe_to_tensors(H_, dataframe.iloc[I])
dataset = data_pipeline.build(H_, tensors)
n = sum(i[0].shape[0] for i in dataset)
round(n / N * 100, 1)

In [ ]:
%%time

N = 10**4

H_ = {
    **H, 
    'epochs': 1,
    'steps_per_epoch': N / H['batch_size'] / 10 * 2,
    'batch_buffer_size': 1,
    'windows_per_chunk': 10,
    'filter_data': True
}

dataframe = initialize.sample_data(H_, sig_data)
print(dataframe.shape)
I = numpy.random.permutation(dataframe.shape[0] // H_['windows_per_chunk'])[:N // H_['windows_per_chunk']]
I = [i*H_['windows_per_chunk'] + j for i in I for j in range(H_['windows_per_chunk'])]
tensors = initialize.dataframe_to_tensors(H_, dataframe.iloc[I])
dataset = data_pipeline.build(H_, tensors)
n = sum(i[0].shape[0] for i in dataset)
round(n / N * 100, 1)

### Check Initial Bias

In [2]:
H = {
    **initialize.load_hypes(),  
    'epochs': 32,
    'window_size': 512,
}
# initialize.run(H)
data = initialize.load_data(H)

Executing op Cast in device /job:localhost/replica:0/task:0/device:CPU:0


In [3]:
for part in ['train', 'validation']:
    data[part] = data_pipeline.build(H, data[part], part)

Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Transpose in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op BroadcastTo in device /job:localhost/replica:0/task:0/device:GPU:0


In [4]:
model = conv_model.build(H)
x, y = next(iter(data['train']))
pressure, gender, died, diagnosis = model.predict_on_batch(x)
display(pressure.numpy().mean(0))
display(gender.numpy().mean())
display(died.numpy().mean())
display(diagnosis.numpy().mean())

Executing op __inference_keras_scratch_graph_721 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_keras_scratch_

array([[127.57433 ,  16.79482 ,  11.543124,  41.37896 ],
       [ 57.895626,   1.865435,   4.496498,  13.72691 ]], dtype=float32)

0.7748717

0.5391463

0.48133767

In [16]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')
died = metadata['death_time'].notna()
died = died.astype('int8').replace({0: -1})
died[metadata['hadm_id'] == -1] = 0
died.value_counts()

 0    1491111
-1     676001
 1     106186
Name: death_time, dtype: int64

In [6]:
# metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')
gender = metadata['gender'].astype(object).replace({'M': 1, 'F': -1})
gender = gender.fillna(0).astype('int8')
gender.value_counts()

 0    1403026
 1     492493
-1     377779
Name: gender, dtype: int64

### Plain Fit

In [5]:
model.fit(
    data['train'],
    validation_data = data['validation'],
    epochs = H['epochs'],
    steps_per_epoch = H['steps_per_epoch'],
    validation_steps = H['validation_steps'],
)

Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Train for 128 steps, validate for 256 steps
Epoch 1/32
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0

KeyboardInterrupt: 

### Overfit Batch

In [3]:
x, y = next(iter(data['train']))
model = conv_model.build(H)
single_batch_repeated = tf.data.Dataset.from_tensors((x, y)).repeat(5000)
model.fit(single_batch_repeated)

Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_628 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op 

KeyboardInterrupt: 

#### Zero Input

In [8]:
%%time

model = conv_model.build(H)
model.fit(data['train'].take(1000).map(lambda x, y: (x*0, y)))

Executing op __inference_keras_scratch_graph_34606 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_34662 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_34667 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_34672 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_34677 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_34773 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_34778 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_35032 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_35037 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_3

W1030 23:01:16.386615 139663798630144 ag_logging.py:146] Entity <function <lambda> at 0x7f050ec4f290> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Unable to locate the source code of <function <lambda> at 0x7f050ec4f290>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code


Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:l

KeyboardInterrupt: 

In [10]:
model.predict(0*x[0:1])

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_63388 in device /job:localhost/replica:0/task:0/device:CPU:0


array([[[126.99957   ,   1.3011948 ,   0.60112923,   2.1011245 ],
        [ 57.99931   ,   0.5009602 ,   0.20098822,   0.7010834 ]]],
      dtype=float32)

#### Simple Conv

In [ ]:
%%time

model = simple_conv_model(H)
for i in range(3):
    data = sample_data(H)
    model.fit(data['train'])
    model.evaluate(data['validation'])

#### Custom Conv

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --host '0.0.0.0'

### Constant Prediction

In [2]:
H = initialize.load_hypes()

path = '/scr-ssd/mimic/initial_data_512.pkl'
with open(path, 'rb') as f:
    data = pickle.load(f)

for k in ['train', 'validation']:
    data[k] = data_pipeline.build(H, data[k], k)

Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GatherV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Transpose in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BroadcastTo in device /job:localhost/replica:0/task:0/device:CPU:0


In [18]:
%%time

n = 10**4
n //= H['batch_size']
zeros = [[0, 0]]*len(H['output_sigs'])
y_ = tf.transpose(tf.constant(zeros, dtype='float32'))
counts = tf.transpose(tf.constant(zeros, dtype='int32'))
for x, y in data['train'].take(n):
    counts += tf.reduce_sum(tf.cast(y != 0, 'int32'), axis=0)
    y_ += tf.reduce_sum(y, axis=0)
y_mean = y_ / tf.cast(counts, dtype='float32')
print(y_mean.numpy())

[[126.72664   18.030378  12.768595  43.229984]
 [ 58.092186   5.902393   6.435025  14.612408]]
CPU times: user 2min 56s, sys: 3.54 s, total: 3min
Wall time: 59.7 s


In [27]:
def loss(H, y_true, y_pred, sample_weight):
    eps = H['relative_target_radius']
    L1 = tf.abs(y_pred - (1 - eps) * y_true)
    L2 = tf.abs(y_pred - (1 + eps) * y_true)
    w1 = [H['loss_weight']['metric'][k] for k in ['systolic', 'diastolic']]
    w2 = [H['loss_weight']['sig'][s] for s in H['output_sigs']]
    w1 = tf.constant(w1, dtype='float32')
    w2 = tf.constant(w2, dtype='float32')
    W = tf.expand_dims(w1, axis=1) * tf.expand_dims(w2, axis=0)
    W = tf.stack([W] * H['batch_size'])
#     W = tf.where(y_true == 0, tf.constant(0, dtype='float32'), W)
    L = tf.reduce_sum(tf.reduce_mean(W * (L1 + L2), axis=0)) / 2
    return L

In [26]:
x = tf.keras.layers.Input(shape=(H['window_size'], len(H['input_sigs_train'])))
z = x[:2, :len(H['output_sigs']), 0]
z *= tf.constant(0, dtype='float32')
z += y_

const_model = tf.keras.models.Model(inputs=x, outputs=z)
const_model.compile(loss=partial(loss, H))
const_model.evaluate(data['validation'].take(n))

Executing op __inference_keras_scratch_graph_2293 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_2298 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_2303 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_2308 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_keras_scratch_graph_2313 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_2409 in device /job:localhost/replica:0/task:0/device:CPU:0
156/156 [==============================] - 84s 537ms/step - loss: 12.4188


12.418806607906635

In [ ]:
%%time

n = 5000
y_ = tf.constant([0, 0], dtype='float32')
for x, y in data['train'].take(n):
    y_ += tf.reduce_mean(y, axis=0)
y_ /= n
print(y_.numpy())

In [ ]:
data_util.calculate_training_speed(H, 5000, seconds_to_train=110)

In [ ]:
df = pandas.DataFrame([
    {'CPU count': 16, 'GPU count': 3, 'Format': 'flac',   'Windows': 10,   'Speed (days/hr)': 14},
    {'CPU count': 16, 'GPU count': 6, 'Format': 'flac',   'Windows': 10,   'Speed (days/hr)': 22},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'flac',   'Windows': 50,   'Speed (days/hr)': 65},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'flac',   'Windows': 1000, 'Speed (days/hr)': 107},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'flac',   'Windows': 1000, 'Speed (days/hr)': 182},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'zlib',   'Windows': 10,   'Speed (days/hr)': 24},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'zlib',   'Windows': 50,   'Speed (days/hr)': 93},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'zlib',   'Windows': 100,  'Speed (days/hr)': 112},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'serial', 'Windows': 10,   'Speed (days/hr)': 68},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'serial', 'Windows': 50,   'Speed (days/hr)': 178},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'serial', 'Windows': 100,  'Speed (days/hr)': 210},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'serial', 'Windows': 1000, 'Speed (days/hr)': 311},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'chunks', 'Windows': 10,   'Speed (days/hr)': 248},
    {'CPU count': 16, 'GPU count': 3, 'Format': 'memory', 'Windows': -1,   'Speed (days/hr)': 345},
    {'CPU count': 16, 'GPU count': 6, 'Format': 'memory', 'Windows': -1,   'Speed (days/hr)': 459},
])
df.sort_values('Speed (days/hr)')

### Dummy Data in Memory

In [ ]:
n_batches = 1000

x = tf.random.uniform(
    shape = [n_batches, H['batch_size'], H['window_size'], len(H['input_sigs'])],
    minval = -1,
    maxval = 1,
)
y = tf.random.uniform(shape=[n_batches, H['batch_size'], 2], minval=40, maxval=200)
dummy_data = tf.data.Dataset.from_tensor_slices((x, y))
model = conv_model.build(H)

%time model.fit(dummy_data, shuffle=False)